This is an extended version of exact_time_inference.jl. It combines it with
Optim + ParameterHandling + Zygote to learn the kernel parameters.
Each of these other packages know nothing about TemporalGPs, they're just general-purpose
packages which play nicely with TemporalGPs (and AbstractGPs).

In [1]:
using AbstractGPs
using TemporalGPs

Load up the separable kernel from TemporalGPs.

In [2]:
using TemporalGPs: RegularSpacing

Load standard packages from the Julia ecosystem

In [3]:
using Optim # Standard optimisation algorithms.
using ParameterHandling # Helper functionality for dealing with model parameters.
using Zygote # Algorithmic Differentiation

Declare model parameters using `ParameterHandling.jl` types.
var_kernel is the variance of the kernel, λ the inverse length scale, and var_noise the
variance of the observation noise. Note that they're all constrained to be positive.

In [4]:
flat_initial_params, unpack = ParameterHandling.value_flatten((
    mean = 3.0,
    var_kernel = positive(0.6),
    λ = positive(0.1),
    var_noise = positive(2.0),
));

Pull out the raw values.

In [5]:
params = unpack(flat_initial_params);

function build_gp(params)
    k = params.var_kernel * Matern52Kernel() ∘ ScaleTransform(params.λ)
    return to_sde(GP(params.mean, k), SArrayStorage(Float64))
end

build_gp (generic function with 1 method)

Specify a collection of inputs. Must be increasing.

In [6]:
T = 1_000_000;
x = RegularSpacing(0.0, 1e-4, T);

Generate some noisy synthetic data from the GP.

In [7]:
f = build_gp(params)
y = rand(f(x, params.var_noise));

Specify an objective function for Optim to minimise in terms of x and y.
We choose the usual negative log marginal likelihood (NLML).

In [8]:
function objective(params)
    f = build_gp(params)
    return -logpdf(f(x, params.var_noise), y)
end

objective (generic function with 1 method)

Optimise using Optim. Zygote takes a little while to compile.

In [9]:
training_results = Optim.optimize(
    objective ∘ unpack,
    θ -> only(Zygote.gradient(objective ∘ unpack, θ)),
    flat_initial_params .+ randn.(), # Perturb the parameters to make learning non-trivial
    BFGS(
        alphaguess = Optim.LineSearches.InitialStatic(scaled=true),
        linesearch = Optim.LineSearches.BackTracking(),
    ),
    Optim.Options(show_trace = true);
    inplace=false,
);

Iter     Function value   Gradient norm 
     0     2.863793e+06     1.874004e+06
 * time: 0.0001380443572998047
     1     1.862721e+06     3.739343e+05
 * time: 126.31552600860596
     2     1.861706e+06     3.755963e+05
 * time: 252.47278904914856
     3     1.846254e+06     3.384893e+05
 * time: 380.4401750564575
     4     1.835583e+06     3.113907e+05
 * time: 509.76091504096985
     5     1.831457e+06     3.005251e+05
 * time: 636.9222149848938
     6     1.829609e+06     2.955744e+05
 * time: 759.0101170539856
     7     1.828288e+06     2.920070e+05
 * time: 873.8118560314178
     8     1.826426e+06     2.869298e+05
 * time: 984.65198802948
     9     1.822313e+06     2.755081e+05
 * time: 1095.8705229759216
    10     1.815872e+06     2.569778e+05
 * time: 1205.8107120990753
    11     1.808565e+06     2.348200e+05
 * time: 1315.834254026413
    12     1.799871e+06     2.064294e+05
 * time: 1429.2335090637207
    13     1.792989e+06     1.818329e+05
 * time: 1541.54181599617


Extracting the final values of the parameters. Should be moderately close to truth.

In [10]:
final_params = unpack(training_results.minimizer)

(mean = 3.1239954837950847, var_kernel = 0.22948768026887953, λ = 0.1293063756483739, var_noise = 2.002348473379149)

Construct the posterior as per usual.

In [11]:
f_final = build_gp(final_params)
f_post = posterior(f_final(x, final_params.var_noise), y);

Specify some locations at which to make predictions.

In [12]:
T_pr = 1_200_000;
x_pr = RegularSpacing(0.0, 1e-4, T_pr);

Compute the exact posterior marginals at `x_pr`.

In [13]:
f_post_marginals = marginals(f_post(x_pr));
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Generate a few posterior samples. Not fantastically-well optimised at present.

In [14]:
f_post_samples = [rand(f_post(x_pr)) for _ in 1:5];

Visualise the posterior. The if block is just to prevent it running in CI.

In [15]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    plt = plot();
    scatter!(plt, x, y; label="", markersize=0.1, alpha=0.1);
    plot!(plt, f_post(x_pr); ribbon_scale=3.0, label="");
    plot!(plt, x_pr, f_post_samples; color=:red, label="");
    savefig(plt, "posterior.png");
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/exact_time_learning/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/exact_time_learning/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;429524aa&#93; Optim v1.7.5
  &#91;2412ca09&#93; ParameterHandling v0.4.6
  &#91;91a5bcdd&#93; Plots v1.38.9
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#564e73f&#96;
  &#91;e88e6eb3&#93; Zygote v0.6.60
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; CPU E5-2673 v4 @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, broadwell&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*